# 8.2. Площади

В этой задаче Вам требуется сравнить значения метрик для 4 различных алгоритмов. Для этого отредактируйте следующий код так, чтобы он соответствовал сформулированному заданию.

In [8]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  precision_recall_curve, auc, roc_auc_score, roc_curve
"""
TODO: make additional imports here
"""

data = fetch_openml(data_id=42608)
X, y = data['data'].drop(columns='Outcome').values, data['data']['Outcome'].astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

sc = StandardScaler()
sc.fit(X_train)
X_train, X_test = sc.transform(X_train), sc.transform(X_test)

"""
In the following part of code specify algorithms with their own parameters by yourself
"""
tree = DecisionTreeClassifier().fit(X_train, y_train)
lr = LogisticRegression().fit(X_train, y_train)
knn = KNeighborsClassifier().fit(X_train, y_train)
svm = SVC(probability=True).fit(X_train, y_train)

def printScore(clf, X_test, y_test):
    y_scores = clf.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_scores)
    auc_pr = auc(recall, precision)

    fpr, tpr, _ = roc_curve(y_test,y_scores)
    auc_roc = auc(fpr, tpr)

    preds = clf.predict(X_test)
    auc_roc_2 = roc_auc_score(y_test, preds)

    print(auc_pr, auc_roc, auc_roc_2)


print('LogisticRegression:', end=' ')
printScore(lr, X_test, y_test)

print('KNeighborsClassifier:', end=' ')
printScore(knn, X_test, y_test)

print('SVC:', end=' ')
printScore(svm, X_test, y_test)

preds = tree.predict(X_test)
auc_roc_2 = roc_auc_score(y_test, preds)
print(auc_roc_2)
precision, recall, _ = precision_recall_curve(y_test, preds)
auc_pr = auc(recall, precision)
print(auc_pr)



LogisticRegression: 0.6639162986466747 0.7980132450331127 0.7098509933774835
KNeighborsClassifier: 0.5876288276242685 0.7504552980132451 0.6627897350993378
SVC: 0.646433465148704 0.7973509933774834 0.7076572847682119
0.7230546357615895
0.7006123124448367


## Примечания

1. Обратите внимание, что StandardScaler может влиять на результаты работы алгоритмов. Поэтому рекомендуем его использовать.

2. Когда мы разбиваем данные на train и test, мы должны понимать, что **все** алгоритмы должны обучаться **только на train**. Test при обучении **не используется**. Более того, в реальной жизни мы вообще ничего не знаем про test. Поэтому StandardScaler нужно обучать только на X_train, а к X_test делать только transform (**не** fit_transform).